In [1]:
import snscrape.modules.twitter as sntwitter
from datetime import date, timedelta
import pandas as pd
import time
import os
import itertools
import numpy as np
print("Done")

Done


In [2]:
df_Cities=pd.read_csv('uscities.csv')
df_Cities.head()
print(df_Cities['state_id'].unique())

['CA' 'WA' 'DE' 'DC' 'WI' 'WV' 'HI' 'FL' 'WY' 'NH' 'NJ' 'NM' 'TX' 'LA'
 'AK' 'NC' 'ND' 'NE' 'TN' 'NY' 'PA' 'RI' 'NV' 'VA' 'CO' 'AL' 'AR' 'VT'
 'IL' 'GA' 'IN' 'IA' 'MA' 'AZ' 'ID' 'CT' 'ME' 'MD' 'OK' 'OH' 'UT' 'MO'
 'MN' 'MI' 'KS' 'MT' 'MS' 'PR' 'SC' 'KY' 'OR' 'SD']


In [3]:
#screening out all cities with more than 100k population 
df_Ci10K=df_Cities[df_Cities['population']>100000]
print(df_Ci10K.shape[0])
print(len(df_Ci10K['state_id'].unique()))#no city in WY is more than 100k population
print(df_Ci10K['state_id'].value_counts())

451
51
CA    88
TX    43
FL    36
NC    16
CO    14
NY    14
WA    14
VA    12
AZ    12
MI    12
IL    11
PA    10
GA     9
OH     9
NV     9
MA     8
SC     8
LA     8
WI     8
TN     8
UT     7
CT     7
OR     7
IN     7
NJ     6
IA     6
AL     5
MN     5
MD     5
KS     5
MO     5
PR     4
OK     4
NM     3
AR     3
ID     3
NH     2
NE     2
WV     2
KY     2
MS     2
RI     1
ME     1
DE     1
DC     1
AK     1
VT     1
ND     1
MT     1
HI     1
SD     1
Name: state_id, dtype: int64


In [4]:
within='15mi'
print(within)
#maxt=10000
# 47 states except CA,TX, FL
us47_1=['WA', 'DE', 'DC', 'WI', 'WV', 'HI', 'NH', 'NJ', 'NM', 'LA', 'AK',
 'NC', 'ND', 'NE', 'TN', 'NY', 'PA', 'RI', 'NV', 'VA', 'CO', 'AL', 'AR', 'VT', 'IL',
 'GA', 'IN', 'IA', 'MA', 'AZ', 'ID', 'CT', 'ME', 'MD', 'OK', 'OH', 'UT', 'MO', 'MN',
 'MI', 'KS', 'MT', 'MS', 'PR', 'SC', 'KY', 'OR', 'SD']
print(us47_1)
columns=['url',
        'date',
                  'content',
                  'tweet.id',
                  'tweet.username', 
                  'tweet.outlinks',
                  'tweet.outlinksss',
                  'tweet.tcooutlinks',
                  'tweet.tcooutlinksss',
                  'lat',
                  'lng',
                  'city',
                  'state_id',
                  'state_name',
                  'cid']
print(columns)

15mi
['WA', 'DE', 'DC', 'WI', 'WV', 'HI', 'NH', 'NJ', 'NM', 'LA', 'AK', 'NC', 'ND', 'NE', 'TN', 'NY', 'PA', 'RI', 'NV', 'VA', 'CO', 'AL', 'AR', 'VT', 'IL', 'GA', 'IN', 'IA', 'MA', 'AZ', 'ID', 'CT', 'ME', 'MD', 'OK', 'OH', 'UT', 'MO', 'MN', 'MI', 'KS', 'MT', 'MS', 'PR', 'SC', 'KY', 'OR', 'SD']
['url', 'date', 'content', 'tweet.id', 'tweet.username', 'tweet.outlinks', 'tweet.outlinksss', 'tweet.tcooutlinks', 'tweet.tcooutlinksss', 'lat', 'lng', 'city', 'state_id', 'state_name', 'cid']


In [5]:
###US states except CA, TX, FL
df_us47_1=pd.DataFrame(columns=columns)

j=1
s=1
for state in us47_1:
    print(20*'--','s=',s,'state=',state,20*'--')

    for i in df_Ci10K[df_Ci10K['state_id']==state].index:

        cood=str(df_Ci10K.loc[i,'lat'])+','+str(df_Ci10K.loc[i,'lng'])+','+str(within)   

        lat=df_Ci10K.loc[i,'lat']
        lng=df_Ci10K.loc[i,'lng']
        city=df_Ci10K.loc[i,'city']
        state_id=df_Ci10K.loc[i,'state_id'],
        state_name=df_Ci10K.loc[i,'state_name']
        cid=df_Ci10K.loc[i,'id']

        lst=[]

        for x, tweet in enumerate(sntwitter.TwitterSearchScraper('(airbnb) OR (hotel) OR (hostel) since:2019-01-01 until:2021-06-30 geocode:"{}"'.format(cood)).get_items()):
            if tweet.id not in df_us47_1['tweet.id'].values:

                lst=[{'url':tweet.url,
                      'date':tweet.date.strftime("%Y-%m-%d %H:%M"),
                      'content':tweet.content,
                      'tweet.id':tweet.id,
                      'tweet.username':tweet.username, 
                      'tweet.outlinks':tweet.outlinks,
                      'tweet.outlinksss':tweet.outlinksss,
                      'tweet.tcooutlinks':tweet.tcooutlinks,
                      'tweet.tcooutlinksss':tweet.tcooutlinksss,
                      'lat':lat,
                      'lng':lng,
                      'city':city,
                      'state_id':state_id,
                      'state_name':state_name,
                      'cid':cid}]
                
                df_us47_1=df_us47_1.append(lst)

        print(i,'\t',df_us47_1.shape[0],'\t',j,'\t',df_Ci10K.loc[i,'city'],'\t',cood,'\t','Done') 

        if (j/3).is_integer():

            df_us47_1.to_csv('text-airbnb+hotel+hostel-tweets_us_2.csv', sep=',', index=False)
            print('top ',j,' cities saved')

        j+=1
    s+=1

df_us47_1.to_csv('text-airbnb+hotel+hostel-tweets_us_2.csv', sep=',', index=False)
print('Done')


---------------------------------------- s= 1 state= WA ----------------------------------------
1 	 4897 	 1 	 Seattle 	 47.6211,-122.3244,15mi 	 Done
15 	 5249 	 2 	 Spokane 	 47.6671,-117.43299999999999,15mi 	 Done
30 	 5348 	 3 	 Kennewick 	 46.1979,-119.1732,15mi 	 Done
top  3  cities saved
33 	 5500 	 4 	 Bremerton 	 47.5436,-122.7121,15mi 	 Done
34 	 6183 	 5 	 Tacoma 	 47.2431,-122.4531,15mi 	 Done
40 	 6428 	 6 	 Olympia 	 47.0417,-122.8959,15mi 	 Done
top  6  cities saved
47 	 9721 	 7 	 Vancouver 	 45.6367,-122.59700000000001,15mi 	 Done
52 	 9992 	 8 	 Marysville 	 48.081,-122.1558,15mi 	 Done
60 	 10364 	 9 	 Bellevue 	 47.5953,-122.155,15mi 	 Done
top  9  cities saved
67 	 10424 	 10 	 Yakima 	 46.5923,-120.5496,15mi 	 Done
68 	 10466 	 11 	 Kent 	 47.3887,-122.2128,15mi 	 Done
71 	 10649 	 12 	 Bellingham 	 48.7543,-122.4687,15mi 	 Done
top  12  cities saved
86 	 10724 	 13 	 Everett 	 47.9524,-122.167,15mi 	 Done
100 	 10724 	 14 	 Renton 	 47.4758,-122.1905,15mi 	 Done

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%28airbnb%29+OR+%28hotel%29+OR+%28hostel%29+since%3A2019-01-01+until%3A2021-06-30+geocode%3A%2240.7161%2C-74.0683%2C15mi%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaAgLXpz9iksB4WhIC-5f2MsJEnEnEVjIRUFYCJehgHREVGQVVMVDUBFbAYFQAA&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='api.twitter.com', port=443): R

3295 	 65374 	 32 	 Jersey City 	 40.7161,-74.0683,15mi 	 Done
3314 	 65438 	 33 	 Newark 	 40.7245,-74.1725,15mi 	 Done
top  33  cities saved
3361 	 68679 	 34 	 Atlantic City 	 39.3797,-74.4527,15mi 	 Done
3463 	 69221 	 35 	 Paterson 	 40.9147,-74.1624,15mi 	 Done
---------------------------------------- s= 9 state= NM ----------------------------------------
3742 	 70318 	 36 	 Albuquerque 	 35.1053,-106.6464,15mi 	 Done
top  36  cities saved
3851 	 70473 	 37 	 Las Cruces 	 32.3265,-106.7893,15mi 	 Done
4102 	 70809 	 38 	 Santa Fe 	 35.6619,-105.9819,15mi 	 Done
---------------------------------------- s= 10 state= LA ----------------------------------------
5885 	 77542 	 39 	 New Orleans 	 30.0687,-89.9288,15mi 	 Done
top  39  cities saved
5942 	 78733 	 40 	 Baton Rouge 	 30.4419,-91.131,15mi 	 Done
5992 	 78860 	 41 	 Metairie 	 29.9976,-90.1781,15mi 	 Done
5995 	 78932 	 42 	 Houma 	 29.5799,-90.7058,15mi 	 Done
top  42  cities saved
6150 	 79040 	 43 	 Monroe 	 32.5183,-92.

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%28airbnb%29+OR+%28hotel%29+OR+%28hostel%29+since%3A2019-01-01+until%3A2021-06-30+geocode%3A%2246.8653%2C-96.8292%2C15mi%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaAwLaB_b34yx4WgsC-qdjOkYsnEnEV3Ox5FYCJehgHREVGQVVMVDUBFRAVAAA%3D&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='api.twitter.com', port=443):

7546 	 88160 	 64 	 Fargo 	 46.8653,-96.8292,15mi 	 Done
---------------------------------------- s= 14 state= NE ----------------------------------------
8204 	 89070 	 65 	 Omaha 	 41.2628,-96.0498,15mi 	 Done
8274 	 89423 	 66 	 Lincoln 	 40.8088,-96.6796,15mi 	 Done
top  66  cities saved
---------------------------------------- s= 15 state= TN ----------------------------------------
8442 	 89581 	 67 	 Kingsport 	 36.5224,-82.5452,15mi 	 Done
8463 	 89685 	 68 	 Clarksville 	 36.5696,-87.3428,15mi 	 Done
8560 	 89931 	 69 	 Murfreesboro 	 35.8493,-86.4117,15mi 	 Done
top  69  cities saved
8569 	 90502 	 70 	 Knoxville 	 35.9692,-83.9496,15mi 	 Done
8636 	 90541 	 71 	 Johnson City 	 36.3406,-82.3803,15mi 	 Done
8656 	 91025 	 72 	 Chattanooga 	 35.0657,-85.2487,15mi 	 Done
top  72  cities saved
8671 	 95664 	 73 	 Nashville 	 36.1715,-86.7843,15mi 	 Done
8810 	 97322 	 74 	 Memphis 	 35.1046,-89.9773,15mi 	 Done
---------------------------------------- s= 16 state= NY ------------

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%28airbnb%29+OR+%28hotel%29+OR+%28hostel%29+since%3A2019-01-01+until%3A2021-06-30+geocode%3A%2240.5834%2C-74.1496%2C15mi%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaCwL2Jv72jlR8WhIC-5f2MsJEnEnEV_JBrFYCJehgHREVGQVVMVDUBFcwJFQAA&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='api.twitter.com', port=443): R

8828 	 102148 	 83 	 Staten Island 	 40.5834,-74.1496,15mi 	 Done
8829 	 102615 	 84 	 Syracuse 	 43.0409,-76.1438,15mi 	 Done
top  84  cities saved
8830 	 102643 	 85 	 Poughkeepsie 	 41.6949,-73.921,15mi 	 Done
8831 	 102783 	 86 	 Yonkers 	 40.9466,-73.8674,15mi 	 Done
8832 	 102802 	 87 	 Binghamton 	 42.1014,-75.9093,15mi 	 Done
top  87  cities saved
8833 	 102920 	 88 	 Utica 	 43.0961,-75.226,15mi 	 Done
---------------------------------------- s= 17 state= PA ----------------------------------------
10109 	 103054 	 89 	 Scranton 	 41.4044,-75.6649,15mi 	 Done
10252 	 103437 	 90 	 Harrisburg 	 40.2752,-76.8843,15mi 	 Done
top  90  cities saved
10478 	 105803 	 91 	 Pittsburgh 	 40.4396,-79.9763,15mi 	 Done
10771 	 105915 	 92 	 Erie 	 42.1168,-80.0733,15mi 	 Done
10834 	 106212 	 93 	 Lancaster 	 40.042,-76.3012,15mi 	 Done
top  93  cities saved
10939 	 106943 	 94 	 Reading 	 40.34,-75.9267,15mi 	 Done
10988 	 107512 	 95 	 Allentown 	 40.5961,-75.4755,15mi 	 Done


Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%28airbnb%29+OR+%28hotel%29+OR+%28hostel%29+since%3A2019-01-01+until%3A2021-06-30+geocode%3A%2240.0077%2C-75.1339%2C15mi%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaAgLa10Nmv0iAWgsC5ibyglpEnEnEV3Mt1FYCJehgHREVGQVVMVDUBFfACFQAA&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='api.twitter.com', port=443): R

11154 	 113408 	 96 	 Philadelphia 	 40.0077,-75.1339,15mi 	 Done
top  96  cities saved
11561 	 113464 	 97 	 York 	 39.9651,-76.7315,15mi 	 Done
11610 	 113544 	 98 	 Pottstown 	 40.2507,-75.6444,15mi 	 Done
---------------------------------------- s= 18 state= RI ----------------------------------------
11641 	 114503 	 99 	 Providence 	 41.823,-71.4187,15mi 	 Done
top  99  cities saved
---------------------------------------- s= 19 state= NV ----------------------------------------
11661 	 140280 	 100 	 Henderson 	 36.0146,-115.0362,15mi 	 Done
11663 	 143059 	 101 	 North Las Vegas 	 36.288000000000004,-115.0901,15mi 	 Done
11681 	 143160 	 102 	 Enterprise 	 36.0163,-115.2207,15mi 	 Done
top  102  cities saved
11695 	 143160 	 103 	 Paradise 	 36.0807,-115.1369,15mi 	 Done
11708 	 144069 	 104 	 Reno 	 39.5497,-119.8483,15mi 	 Done
11710 	 144069 	 105 	 Spring Valley 	 36.0987,-115.2619,15mi 	 Done
top  105  cities saved
11718 	 144090 	 106 	 Sparks 	 39.5729,-119.7157,15mi 	 D

KeyboardInterrupt: 

In [20]:
#print(df_us47_1['city'].unique())
#df_us47_2=df_us47_1[df_us47_1.city !='Avondale']
#print(df_us47_2.shape)
#df_us47_2.to_csv('text-airbnb+hotel+hoster-tweets_us47_1.csv', sep=',', index=False)
#print('Done')

0     Seattle
0     Seattle
0     Seattle
0     Seattle
0     Seattle
       ...   
0    Avondale
0    Avondale
0    Avondale
0    Avondale
0    Avondale
Name: city, Length: 176278, dtype: object
(175355, 15)
Done


In [5]:
df_us47_2=pd.read_csv('text-airbnb+hotel+hostel-tweets_us_2.csv')
df_us47_2.shape

(144094, 15)

In [6]:
#continue scraping from VA
us47_2=[#'WA', 'DE', 'DC', 'WI', 'WV', 'HI', 'NH', 'NJ', 'NM', 'LA', 'AK','NC', 'ND', 'NE', 'TN', 'NY', 'PA', 'RI', 'NV', 
    'VA', 'CO', 'AL', 'AR', 'VT', 'IL',
 'GA', 'IN', 'IA', 'MA', 'AZ', 'ID', 'CT', 'ME', 'MD', 'OK', 'OH', 'UT', 'MO', 'MN',
 'MI', 'KS', 'MT', 'MS', 'PR', 'SC', 'KY', 'OR', 'SD']
print(us47_2)

['VA', 'CO', 'AL', 'AR', 'VT', 'IL', 'GA', 'IN', 'IA', 'MA', 'AZ', 'ID', 'CT', 'ME', 'MD', 'OK', 'OH', 'UT', 'MO', 'MN', 'MI', 'KS', 'MT', 'MS', 'PR', 'SC', 'KY', 'OR', 'SD']


In [7]:
j=109
s=20
for state in us47_2:
    print(20*'--','s=',s,'state=',state,20*'--')

    for i in df_Ci10K[df_Ci10K['state_id']==state].index:

        cood=str(df_Ci10K.loc[i,'lat'])+','+str(df_Ci10K.loc[i,'lng'])+','+str(within)   

        lat=df_Ci10K.loc[i,'lat']
        lng=df_Ci10K.loc[i,'lng']
        city=df_Ci10K.loc[i,'city']
        state_id=df_Ci10K.loc[i,'state_id'],
        state_name=df_Ci10K.loc[i,'state_name']
        cid=df_Ci10K.loc[i,'id']

        lst=[]

        for x, tweet in enumerate(sntwitter.TwitterSearchScraper('(airbnb) OR (hotel) OR (hostel) since:2019-01-01 until:2021-06-30 geocode:"{}"'.format(cood)).get_items()):
            if tweet.id not in df_us47_2['tweet.id'].values:

                lst=[{'url':tweet.url,
                      'date':tweet.date.strftime("%Y-%m-%d %H:%M"),
                      'content':tweet.content,
                      'tweet.id':tweet.id,
                      'tweet.username':tweet.username, 
                      'tweet.outlinks':tweet.outlinks,
                      'tweet.outlinksss':tweet.outlinksss,
                      'tweet.tcooutlinks':tweet.tcooutlinks,
                      'tweet.tcooutlinksss':tweet.tcooutlinksss,
                      'lat':lat,
                      'lng':lng,
                      'city':city,
                      'state_id':state_id,
                      'state_name':state_name,
                      'cid':cid}]
                
                df_us47_2=df_us47_2.append(lst)

        print(i,'\t',df_us47_2.shape[0],'\t',j,'\t',df_Ci10K.loc[i,'city'],'\t',cood,'\t','Done') 

        if (j/3).is_integer():

            df_us47_2.to_csv('text-airbnb+hotel+hostel-tweets_us_2.csv', sep=',', index=False)
            print('top ',j,' cities saved')

        j+=1
    s+=1

df_us47_2.to_csv('text-airbnb+hotel+hostel-tweets_us_2.csv', sep=',', index=False)
print('Done')


---------------------------------------- s= 20 state= VA ----------------------------------------
11876 	 145056 	 109 	 Chesapeake 	 36.6778,-76.3024,15mi 	 Done
11974 	 145498 	 110 	 Newport News 	 37.1051,-76.5185,15mi 	 Done
12081 	 146096 	 111 	 Norfolk 	 36.8945,-76.259,15mi 	 Done
top  111  cities saved
12088 	 146371 	 112 	 Roanoke 	 37.2785,-79.958,15mi 	 Done
12090 	 146533 	 113 	 Alexandria 	 38.8185,-77.0861,15mi 	 Done
12112 	 146659 	 114 	 Charlottesville 	 38.0375,-78.4855,15mi 	 Done
top  114  cities saved


Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%28airbnb%29+OR+%28hotel%29+OR+%28hostel%29+since%3A2019-01-01+until%3A2021-06-30+geocode%3A%2238.8786%2C-77.1011%2C15mi%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaAgLaV69ilniAWisCw8aGBqJEnEnEViNFvFYCJehgHREVGQVVMVDUBFdwGFQAA&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: non-200 status code, retrying


12126 	 146855 	 115 	 Arlington 	 38.8786,-77.1011,15mi 	 Done
12199 	 146942 	 116 	 Fredericksburg 	 38.2992,-77.4872,15mi 	 Done
12220 	 146953 	 117 	 Hampton 	 37.0551,-76.3629,15mi 	 Done
top  117  cities saved
12233 	 148120 	 118 	 Richmond 	 37.5295,-77.4756,15mi 	 Done
12274 	 148223 	 119 	 Lynchburg 	 37.4003,-79.1909,15mi 	 Done
12320 	 148224 	 120 	 Virginia Beach 	 36.7335,-76.0435,15mi 	 Done
top  120  cities saved
---------------------------------------- s= 21 state= CO ----------------------------------------
12377 	 151633 	 121 	 Aurora 	 39.7084,-104.7274,15mi 	 Done
12400 	 152148 	 122 	 Denver 	 39.7621,-104.8759,15mi 	 Done
12411 	 152252 	 123 	 Grand Junction 	 39.0877,-108.5673,15mi 	 Done
top  123  cities saved
12412 	 152438 	 124 	 Thornton 	 39.9205,-104.9443,15mi 	 Done
12487 	 153127 	 125 	 Colorado Springs 	 38.8674,-104.7606,15mi 	 Done
12602 	 153299 	 126 	 Lakewood 	 39.6977,-105.1168,15mi 	 Done
top  126  cities saved
12604 	 153348 	 127 	 Pu

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%28airbnb%29+OR+%28hotel%29+OR+%28hostel%29+since%3A2019-01-01+until%3A2021-06-30+geocode%3A%2241.8373%2C-87.6862%2C15mi%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaKwL39vp3I7iMWhICzyejrs5EnEnEVrKh3FYCJehgHREVGQVVMVDUBFeABFQAA&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: non-200 status code, retrying
Error retrieving https://api.twitter.com/2/search/adapti

16289 	 171112 	 152 	 Chicago 	 41.8373,-87.6862,15mi 	 Done
16672 	 171163 	 153 	 Peoria 	 40.7521,-89.6155,15mi 	 Done
top  153  cities saved
16836 	 171172 	 154 	 Joliet 	 41.5189,-88.1499,15mi 	 Done
---------------------------------------- s= 26 state= GA ----------------------------------------
17000 	 171332 	 155 	 Columbus 	 32.51,-84.8771,15mi 	 Done
17091 	 171372 	 156 	 Macon 	 32.8065,-83.6974,15mi 	 Done
top  156  cities saved
17334 	 171552 	 157 	 Athens 	 33.9508,-83.3689,15mi 	 Done
17350 	 171601 	 158 	 Gainesville 	 34.2902,-83.8303,15mi 	 Done
17426 	 171622 	 159 	 Warner Robins 	 32.5975,-83.6526,15mi 	 Done
top  159  cities saved
17464 	 180444 	 160 	 Atlanta 	 33.7627,-84.4225,15mi 	 Done
17467 	 180722 	 161 	 Augusta 	 33.3645,-82.0708,15mi 	 Done
17475 	 181575 	 162 	 Savannah 	 32.0281,-81.1785,15mi 	 Done
top  162  cities saved
17476 	 182254 	 163 	 Sandy Springs 	 33.9366,-84.3703,15mi 	 Done
---------------------------------------- s= 27 state= I

In [8]:
df_us47_2.shape

(243858, 15)

In [9]:
#continue scraping CA TX FL
us47_3=['CA','TX','FL']
print(us47_3)

['CA', 'TX', 'FL']


In [10]:
j=285
s=49
for state in us47_3:
    print(20*'--','s=',s,'state=',state,20*'--')

    for i in df_Ci10K[df_Ci10K['state_id']==state].index:

        cood=str(df_Ci10K.loc[i,'lat'])+','+str(df_Ci10K.loc[i,'lng'])+','+str(within)   

        lat=df_Ci10K.loc[i,'lat']
        lng=df_Ci10K.loc[i,'lng']
        city=df_Ci10K.loc[i,'city']
        state_id=df_Ci10K.loc[i,'state_id'],
        state_name=df_Ci10K.loc[i,'state_name']
        cid=df_Ci10K.loc[i,'id']

        lst=[]

        for x, tweet in enumerate(sntwitter.TwitterSearchScraper('(airbnb) OR (hotel) OR (hostel) since:2019-01-01 until:2021-06-30 geocode:"{}"'.format(cood)).get_items()):
            if tweet.id not in df_us47_2['tweet.id'].values:

                lst=[{'url':tweet.url,
                      'date':tweet.date.strftime("%Y-%m-%d %H:%M"),
                      'content':tweet.content,
                      'tweet.id':tweet.id,
                      'tweet.username':tweet.username, 
                      'tweet.outlinks':tweet.outlinks,
                      'tweet.outlinksss':tweet.outlinksss,
                      'tweet.tcooutlinks':tweet.tcooutlinks,
                      'tweet.tcooutlinksss':tweet.tcooutlinksss,
                      'lat':lat,
                      'lng':lng,
                      'city':city,
                      'state_id':state_id,
                      'state_name':state_name,
                      'cid':cid}]
                
                df_us47_2=df_us47_2.append(lst)

        print(i,'\t',df_us47_2.shape[0],'\t',j,'\t',df_Ci10K.loc[i,'city'],'\t',cood,'\t','Done') 

        if (j/3).is_integer():

            df_us47_2.to_csv('text-airbnb+hotel+hostel-tweets_us_2.csv', sep=',', index=False)
            print('top ',j,' cities saved')

        j+=1
    s+=1

df_us47_2.to_csv('text-airbnb+hotel+hostel-tweets_us_2.csv', sep=',', index=False)
print('Done')


---------------------------------------- s= 49 state= CA ----------------------------------------
0 	 259614 	 285 	 Los Angeles 	 34.1139,-118.4068,15mi 	 Done
top  285  cities saved
2 	 259876 	 286 	 San Francisco 	 37.7562,-122.443,15mi 	 Done
3 	 264438 	 287 	 San Diego 	 32.8312,-117.1225,15mi 	 Done
4 	 264471 	 288 	 Riverside 	 33.9381,-117.3948,15mi 	 Done
top  288  cities saved
5 	 264800 	 289 	 Sacramento 	 38.5667,-121.4683,15mi 	 Done
6 	 264900 	 290 	 San Jose 	 37.3021,-121.8489,15mi 	 Done
7 	 265240 	 291 	 Fresno 	 36.7831,-119.7941,15mi 	 Done
top  291  cities saved
8 	 265254 	 292 	 Concord 	 37.9722,-122.0016,15mi 	 Done
9 	 265280 	 293 	 Mission Viejo 	 33.6095,-117.655,15mi 	 Done
10 	 265430 	 294 	 Bakersfield 	 35.353,-119.0359,15mi 	 Done
top  294  cities saved
11 	 265438 	 295 	 Murrieta 	 33.571999999999996,-117.1909,15mi 	 Done
12 	 265582 	 296 	 Long Beach 	 33.798,-118.1675,15mi 	 Done
13 	 266960 	 297 	 Oakland 	 37.7903,-122.2165,15mi 	 Done
t

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%28airbnb%29+OR+%28hotel%29+OR+%28hostel%29+since%3A2019-01-01+until%3A2021-06-30+geocode%3A%2234.1818%2C-118.2468%2C15mi%22&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaAwKvZxPqTyCUWgMCi4ab8o5EnEnEV5OB2FYCJehgHREVGQVVMVDUBFY4CFQAA&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='api.twitter.com', port=443): 

38 	 275291 	 318 	 Glendale 	 34.1818,-118.2468,15mi 	 Done
top  318  cities saved
39 	 275292 	 319 	 Huntington Beach 	 33.696,-118.0025,15mi 	 Done
41 	 275590 	 320 	 Salinas 	 36.6884,-121.6317,15mi 	 Done
42 	 275610 	 321 	 Santa Cruz 	 36.9789,-122.0346,15mi 	 Done
top  321  cities saved
43 	 275625 	 322 	 Rancho Cucamonga 	 34.1248,-117.5666,15mi 	 Done
44 	 275644 	 323 	 Hemet 	 33.7341,-116.9969,15mi 	 Done
45 	 275682 	 324 	 Oceanside 	 33.2247,-117.3083,15mi 	 Done
top  324  cities saved
46 	 275682 	 325 	 Ontario 	 34.0393,-117.6064,15mi 	 Done
48 	 275703 	 326 	 Garden Grove 	 33.7787,-117.9601,15mi 	 Done
49 	 275705 	 327 	 Vallejo 	 38.1133,-122.2359,15mi 	 Done
top  327  cities saved
50 	 275722 	 328 	 Elk Grove 	 38.416,-121.384,15mi 	 Done
51 	 275812 	 329 	 Corona 	 33.861999999999995,-117.5641,15mi 	 Done
53 	 275863 	 330 	 Hayward 	 37.6328,-122.0772,15mi 	 Done
top  330  cities saved
54 	 275863 	 331 	 Palmdale 	 34.5944,-118.1057,15mi 	 Done
55 	 275

['TX', 'FL']
